In [ ]:
"""
Read ModEM model, ModEM's Jacobian.

Model order reduction by discreste (co)sinus transform.

@author: vrath Feb 2021

"""
import os
import sys
from sys import exit as error
import time
import warnings

In [ ]:
import numpy as np
import math as ma
import netCDF4 as nc
import scipy.ndimage as spn
import scipy.linalg as spl

In [ ]:
import vtk
import pyvista as pv
import PVGeo as pvg

In [ ]:
mypath = ["/home/vrath/Py4MT/py4mt/modules/", "/home/vrath/Py4MT/py4mt/scripts/"]
for pth in mypath:
    if pth not in sys.path:
        sys.path.append(pth)

In [ ]:
import modules
import modem as mod
import util as utl

In [ ]:
warnings.simplefilter(action="ignore", category=FutureWarning)